In [1]:
using Plots
using CSV
using DataFrames
using Statistics
using StatsPlots
using Printf
# Store Data
using DelimitedFiles

$$
D = \frac{mean(dx^2) + mean(dy^2)}{4\times Times}
$$

In [2]:
function plotData(path, methods, savepath) # Loads Data, alpha is horizontal beta is vertical

    # Initiation
    alpha_max = 0.8
    alpha_min = 0.0
    alpha_step = 0.2
    alpha_nStep = size(alpha_min:alpha_step:alpha_max, 1)
    beta_max = 0.8
    beta_min = 0.0
    beta_step = 0.2
    beta_nStep = size(beta_min:beta_step:beta_max, 1)
    
    raw = Matrix{Matrix{BigFloat}}(undef, beta_nStep, alpha_nStep)

    # Load data
    for b in 1:beta_nStep
        for a in 1:alpha_nStep
            alpha = round((a-1) * alpha_step; digits = 1)
            beta = round((b-1) * beta_step; digits = 1)
            raw[b, a] = Matrix{BigFloat}(DataFrame(CSV.File("$path$methods-data-alpha=$alpha-beta=$beta.csv", header=false)))
        end
    end
    
    # Process
    time_step = size(raw[1, 1], 1)
    Times = 0.2*time_step
    
    DiffusionCoefficient = Matrix{BigFloat}(undef, beta_nStep, alpha_nStep)
    
    for b in 1:beta_nStep
        for a in 1:alpha_nStep
            DiffusionCoefficient[b, a] = (mean(raw[b, a][:,1].^2) + mean(raw[b, a][:,2].^2)) / (4 * Times)
        end
    end
    
    # Creating a DataFrame with row and column headers
    df = DataFrame(DiffusionCoefficient, ["alpha=$i" for i in alpha_min:alpha_step:alpha_max])
    df[!, :beta] = ["beta=$i" for i in beta_min:beta_step:beta_max]
    writedlm("$savepath$methods-analysis-DiffusionCoefficient.csv", DiffusionCoefficient, ',', header=true);
    
    # Plots Diffusion Coefficient
    p1 = plot(grid=true, legend=true, title="control beta")
    for b in 1:beta_nStep
        beta = round((b - 1) * beta_step, digits=1)
        plot!([i for i in alpha_min:alpha_step:alpha_max], DiffusionCoefficient[b, :], label="beta=$beta")
    end
    savefig(p1, "$savepath$methods-analysis-LineBeta.png")
    
    p2 = plot(grid=true, legend=true, title="control alpha")
    for a in 1:alpha_nStep
        alpha = round((a - 1) * alpha_step, digits=1)
        plot!([i for i in beta_min:beta_step:beta_max], DiffusionCoefficient[:, a], label="alpha=$alpha")
    end
    savefig(p2, "$savepath$methods-analysis-LineAlpha.png")
    
    p3 = heatmap(["alpha=$i" for i in alpha_min:alpha_step:alpha_max], ["beta=$i" for i in beta_min:beta_step:beta_max], log.(DiffusionCoefficient), xlabel="alpha", ylabel="beta", title="Diffusion Coefficient Log Heatmap", c=:viridis, color=:auto)
    savefig(p3, "$savepath$methods-analysis-LogHeatMap.png")

    # Plots histogram
    # display(histogram(raw[1, 1][:, 1], bins=200, color="red", label="dx", alpha=0.5, xlabel="Displacement", ylabel="Frequency", title="Displacement ", size=(1920,1080)))
    # display(histogram(raw[1, 1][:, 2], bins=200, color="green", label="dy", alpha=0.5, xlabel="Displacement", ylabel="Frequency", title="Displacement ", size=(1920,1080)))
    # display(histogram(raw[1, 1][:, 3], bins=200, color="blue", label="dtheta", alpha=0.5, xlabel="Displacement", ylabel="Frequency", title="Displacement ", size=(1920,1080), polar=true))

    plots = Array{Any}(undef, beta_nStep, alpha_nStep)

    for b in 1:beta_nStep
        for a in 1:alpha_nStep
            alpha = round((a - 1) * alpha_step, digits=1)
            beta = round((b - 1) * beta_step, digits=1)
            
            q25, q75 = quantile(raw[b, a][:, 1], [0.25, 0.75])
            
            mean_str = @sprintf("%.2e", mean(raw[b, a][:, 1]))
            median_str = @sprintf("%.2e", median(raw[b, a][:, 1]))
            q25_str = @sprintf("%.2e", q25)
            q75_str = @sprintf("%.2e", q75)
            
            p = plot(xlabel="Displacement", ylabel="Frequency", title="dx,m$mean_str,md$median_str,q25=$q25_str,q75=$q75_str", size=(4000, 2000))
            
            histogram!(raw[b, a][:, 1], bins=200, color="red", label="dx", alpha=0.5)
            boxplot!(raw[b, a][:, 1], seriestype=:box, orientation=:h, whisker_width=10, label="")

            plots[b, a] = p
        end
    end
    
    p4 = plot(plots..., layout=(beta_nStep, alpha_nStep), polar=true)
    savefig(p4, "$savepath$methods-analysis-DisplacementSummary_dx.png")
    
    for b in 1:beta_nStep
        for a in 1:alpha_nStep
            alpha = round((a - 1) * alpha_step, digits=1)
            beta = round((b - 1) * beta_step, digits=1)
            
            q25, q75 = quantile(raw[b, a][:, 2], [0.25, 0.75])
            
            mean_str = @sprintf("%.2e", mean(raw[b, a][:, 2]))
            median_str = @sprintf("%.2e", median(raw[b, a][:, 2]))
            q25_str = @sprintf("%.2e", q25)
            q75_str = @sprintf("%.2e", q75)
            
            p = plot(xlabel="Displacement", ylabel="Frequency", title="dy,m$mean_str,md$median_str,q25=$q25_str,q75=$q75_str", size=(4000, 2000))
            
            histogram!(raw[b, a][:, 2], bins=200, color="red", label="dy", alpha=0.5)
            boxplot!(raw[b, a][:, 2], seriestype=:box, orientation=:h, whisker_width=10, label="")

            plots[b, a] = p
        end
    end
    
    p5 = plot(plots..., layout=(beta_nStep, alpha_nStep), polar=true)
    savefig(p5, "$savepath$methods-analysis-DisplacementSummary_dy.png")
    
    
    for b in 1:beta_nStep
        for a in 1:alpha_nStep
            alpha = round((a - 1) * alpha_step, digits=1)
            beta = round((b - 1) * beta_step, digits=1)
            
            q25, q75 = quantile(raw[b, a][:, 3], [0.25, 0.75])
            
            mean_str = @sprintf("%.2e", mean(raw[b, a][:, 3]))
            median_str = @sprintf("%.2e", median(raw[b, a][:, 3]))
            q25_str = @sprintf("%.2e", q25)
            q75_str = @sprintf("%.2e", q75)
            
            p = plot(xlabel="Displacement", ylabel="Frequency", title="dtheta,m$mean_str,md$median_str,q25=$q25_str,q75=$q75_str", size=(4000, 2000))
            
            histogram!(raw[b, a][:, 3], bins=200, color="red", label="dtheta", alpha=0.5)
            boxplot!(raw[b, a][:, 3], seriestype=:box, orientation=:h, whisker_width=10, label="")

            plots[b, a] = p
        end
    end
    
    p6 = plot(plots..., layout=(beta_nStep, alpha_nStep), polar=true)
    savefig(p6, "$savepath$methods-analysis-DisplacementSummary_dtheta.png")
    
    # display(p1)
    # display(p2)
    # display(p3)
    # display(p4)
    # display(p5)
    # display(p6)
end

plotData (generic function with 1 method)

In [3]:
plotData("data/VDisjoint2000/", "VDisjoint", "data/Analysis/")

┌ Warning: No strict ticks found
└ @ PlotUtils C:\Users\yuang\.julia\packages\PlotUtils\jEGKP\src\ticks.jl:191
┌ Warning: No strict ticks found
└ @ PlotUtils C:\Users\yuang\.julia\packages\PlotUtils\jEGKP\src\ticks.jl:191
┌ Warning: No strict ticks found
└ @ PlotUtils C:\Users\yuang\.julia\packages\PlotUtils\jEGKP\src\ticks.jl:191
┌ Warning: No strict ticks found
└ @ PlotUtils C:\Users\yuang\.julia\packages\PlotUtils\jEGKP\src\ticks.jl:191


"C:\\Users\\yuang\\temp\\rex_iav_diffusion\\data\\Analysis\\VDisjoint-analysis-DisplacementSummary_dtheta.png"

In [4]:
plotData("data/Mixing1000/", "Mixing", "data/Analysis/")

┌ Warning: No strict ticks found
└ @ PlotUtils C:\Users\yuang\.julia\packages\PlotUtils\jEGKP\src\ticks.jl:191
┌ Warning: No strict ticks found
└ @ PlotUtils C:\Users\yuang\.julia\packages\PlotUtils\jEGKP\src\ticks.jl:191
┌ Warning: No strict ticks found
└ @ PlotUtils C:\Users\yuang\.julia\packages\PlotUtils\jEGKP\src\ticks.jl:191
┌ Warning: No strict ticks found
└ @ PlotUtils C:\Users\yuang\.julia\packages\PlotUtils\jEGKP\src\ticks.jl:191


"C:\\Users\\yuang\\temp\\rex_iav_diffusion\\data\\Analysis\\Mixing-analysis-DisplacementSummary_dtheta.png"

In [5]:
plotData("data/Random180/", "Random", "data/Analysis/")

┌ Warning: No strict ticks found
└ @ PlotUtils C:\Users\yuang\.julia\packages\PlotUtils\jEGKP\src\ticks.jl:191
┌ Warning: No strict ticks found
└ @ PlotUtils C:\Users\yuang\.julia\packages\PlotUtils\jEGKP\src\ticks.jl:191
┌ Warning: No strict ticks found
└ @ PlotUtils C:\Users\yuang\.julia\packages\PlotUtils\jEGKP\src\ticks.jl:191
┌ Warning: No strict ticks found
└ @ PlotUtils C:\Users\yuang\.julia\packages\PlotUtils\jEGKP\src\ticks.jl:191


"C:\\Users\\yuang\\temp\\rex_iav_diffusion\\data\\Analysis\\Random-analysis-DisplacementSummary_dtheta.png"